In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
POSSIBLE_VALUES = [2,3,4,5,6,7,8,9,10,11]

DECK_COLUMNS = [str(i) for i in POSSIBLE_VALUES]
HAND_COLUMNS = ["chance", "score", "cards", "aces"] + DECK_COLUMNS

In [3]:
def get_deck(number_decks=6):
    deck = pd.Series([number_decks*4]*len(DECK_COLUMNS) ,index=DECK_COLUMNS)
    deck.loc["10"] *= 4
    return deck

In [4]:
def get_hand(deck=None, card_values=[], deck_update=True):
    if deck is None:
        deck = get_deck()
    hand = pd.DataFrame([1.0, 0, 0, 0] + deck.values.flatten().tolist(), index=HAND_COLUMNS).T

    for card_value in card_values:
        # updating score
        hand["score"] += card_value
        # updating cards
        hand["cards"] += 1
        # updating aces
        hand["aces"] += (card_value == 11)

        # if necessary, using ace as 1 instead of 11
        acenecessary = (hand["score"] > 21) & (hand["aces"] > 0)
        hand.loc[acenecessary,"score"] -= 10
        hand.loc[acenecessary,"aces"] -= 1
        if deck_update:
            hand[f"{card_value}"] -= 1
    return hand

In [5]:
def get_hand_hitting_once(hand):
    """get updated hand when hitting once"""

    assert type(hand) == pd.DataFrame and hand.columns.tolist() == HAND_COLUMNS, f"hand is not valid, got {hand}"

    busted = hand["score"] > 21
    if busted.sum() == len(hand):
        print("All possible hands are busted")
        return hand
    elif busted.sum() > 0:
        print(f"{busted.sum()} possible hands are busted")
    
    possible_outcomes = pd.DataFrame(hand[busted], columns=HAND_COLUMNS)
    hand = hand[~busted]

    for possible_value in POSSIBLE_VALUES:
        new_possibles = hand.copy()
        # updating chance: multiplying by the probability of drawing that value
        new_possibles["chance"] *= (new_possibles[f"{possible_value}"]/new_possibles[DECK_COLUMNS].sum(axis=1))
        # updating score
        new_possibles["score"] += possible_value
        # updating cards
        new_possibles["cards"] += 1
        # updating aces
        new_possibles["aces"] += (possible_value == 11)

        # if necessary, using ace as 1 instead of 11
        acenecessary = (new_possibles["score"] > 21) & (new_possibles["aces"] > 0)
        new_possibles.loc[acenecessary,"score"] -= 10
        new_possibles.loc[acenecessary,"aces"] -= 1
        new_possibles[f"{possible_value}"] -= 1

        # adding to all possible outcomes
        possible_outcomes = pd.concat((possible_outcomes, new_possibles), axis=0, ignore_index=True)
    return possible_outcomes

In [6]:
def get_score(hand):
    """get score of a hand"""
    assert type(hand) == pd.DataFrame and hand.columns.tolist() == HAND_COLUMNS, f"hand is not valid, got {hand}"
    assert len(hand) == 1, f"hand has more than one possible outcome, got {len(hand)}"
    assert hand[0,"chance"] > 0.9999999999999999, f"hand has chance less than 1, got {hand}"

    hand = hand[0]
    
    # checking blackjack
    if (hand["score"] == 21) and (hand["cards"] == 2):
        return 22
    
    # checking busted
    if hand["score"] > 21:
        return 0

    return hand["score"]
    

In [7]:
def get_score_probabilities(hand):
    """get probabilities of each score in a hand"""

    assert type(hand) == pd.DataFrame and hand.columns.tolist() == HAND_COLUMNS, f"hand is not valid, got {hand}"

    valcounts = pd.Series(data=np.zeros(35), index=np.arange(35))

    # counting blackjacks
    probability_of_blackjack = hand[(hand["score"] == 21) & (hand["cards"] == 2)]["chance"].sum()

    # grouping vals
    grouped_df = pd.DataFrame(hand[["score","chance"]].values, columns=["score","chance"]).groupby("score").sum()
    valcounts.loc[grouped_df.index.values.astype(int)] = grouped_df["chance"].values

    # adding busts as 0
    valcounts.loc[0] = valcounts[valcounts.index > 21].sum()
    valcounts = valcounts[valcounts.index <= 21].sort_index(ascending=True)

    # adding blackjacks
    valcounts.loc[21] -= probability_of_blackjack
    valcounts.loc[22] = probability_of_blackjack

    return valcounts

In [8]:
deck = get_deck()
deck

2     24
3     24
4     24
5     24
6     24
7     24
8     24
9     24
10    96
11    24
dtype: int64

In [9]:
hand = get_hand(deck, [])
hand

,chance,score,cards,aces,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,0.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,96.0,24.0


In [10]:
hand = get_hand_hitting_once(hand)
hand

,chance,score,cards,aces,2,3,4,5,6,7,8,9,10,11
0,0.076923,2.0,1.0,0.0,23.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,96.0,24.0
1,0.076923,3.0,1.0,0.0,24.0,23.0,24.0,24.0,24.0,24.0,24.0,24.0,96.0,24.0
2,0.076923,4.0,1.0,0.0,24.0,24.0,23.0,24.0,24.0,24.0,24.0,24.0,96.0,24.0
3,0.076923,5.0,1.0,0.0,24.0,24.0,24.0,23.0,24.0,24.0,24.0,24.0,96.0,24.0
4,0.076923,6.0,1.0,0.0,24.0,24.0,24.0,24.0,23.0,24.0,24.0,24.0,96.0,24.0
5,0.076923,7.0,1.0,0.0,24.0,24.0,24.0,24.0,24.0,23.0,24.0,24.0,96.0,24.0
6,0.076923,8.0,1.0,0.0,24.0,24.0,24.0,24.0,24.0,24.0,23.0,24.0,96.0,24.0
7,0.076923,9.0,1.0,0.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,23.0,96.0,24.0
8,0.307692,10.0,1.0,0.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,95.0,24.0
9,0.076923,11.0,1.0,1.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,96.0,23.0


In [11]:
scores = get_score_probabilities(hand)
scores

0     0.000000
1     0.000000
2     0.076923
3     0.076923
4     0.076923
5     0.076923
6     0.076923
7     0.076923
8     0.076923
9     0.076923
10    0.307692
11    0.076923
12    0.000000
13    0.000000
14    0.000000
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
20    0.000000
21    0.000000
22    0.000000
dtype: float64

In [12]:
def get_dealer_score_probabilities(deck, dealer_hand):
    """get probabilities of each possible score given the dealers current hand"""

    assert type(dealer_hand) == pd.DataFrame and dealer_hand.columns.tolist() == HAND_COLUMNS, f"hand is not valid, got {dealer_hand}"

    
    final = dealer_hand[dealer_hand["score"] >= 17]
    playable = dealer_hand[dealer_hand["score"] < 17]

    while len(playable) > 0:
        new_dealer_hand = get_hand_hitting_once(playable)
        final = pd.concat((final, new_dealer_hand[new_dealer_hand["score"] >= 17]), axis=0, ignore_index=True)
        playable = new_dealer_hand[new_dealer_hand["score"] < 17]
    
    scores = get_score_probabilities(final)
    
    return scores

In [13]:
def get_expected_return_standing(deck, player_hand, dealer_hand):
    """Returns the expected return of standing with the given cards."""

    assert type(player_hand) == pd.DataFrame and player_hand.columns.tolist() == HAND_COLUMNS, f"player_hand is not valid, got {player_hand}"
    assert type(dealer_hand) == pd.DataFrame and dealer_hand.columns.tolist() == HAND_COLUMNS, f"dealer_hand is not valid, got {dealer_hand}"
    assert all(dealer_hand["cards"] == 1), "Dealer must have exactly one card showing."

    print("Calculating expected return of standing with the following hand:")
    print("player", player_hand)
    print("dealer", dealer_hand)


    dealer_score_probabilities = get_dealer_score_probabilities(deck, dealer_hand)
    player_score_probabilities = get_score_probabilities(player_hand)

    print("Dealer score probabilities:")
    print(dealer_score_probabilities)

    win, push, lose = 0.0, 0.0, 0.0

    for score, chance in player_score_probabilities.items():
        win += chance * dealer_score_probabilities[dealer_score_probabilities.index < score].sum()
        push += chance * dealer_score_probabilities[dealer_score_probabilities.index == score].sum()
        lose += chance * dealer_score_probabilities[dealer_score_probabilities.index > score].sum()

    print(pd.DataFrame([win,push,lose], index=["win","push","lose"], columns=["probability"]).T)
    print("Expected return of standing is", win - lose)

    return win - lose

In [14]:
def get_expected_return_hitting_once(deck, player_hand, dealer_hand):
    """Returns the expected return of hitting exactly once with the given cards."""
    
    assert type(player_hand) == pd.DataFrame and player_hand.columns.tolist() == HAND_COLUMNS, f"player_hand is not valid, got {player_hand}"
    assert type(dealer_hand) == pd.DataFrame and dealer_hand.columns.tolist() == HAND_COLUMNS, f"dealer_hand is not valid, got {dealer_hand}"
    assert all(dealer_hand["cards"] == 1), "Dealer must have exactly one card showing."

    print("Calculating expected return of standing with the following hand:")
    print("player", player_hand)
    print("dealer", dealer_hand)


    dealer_score_probabilities = get_dealer_score_probabilities(deck, dealer_hand)
    # TODO: this is actually incorrect, as it does not take into account the card just drawn by the player

    player_hand = get_hand_hitting_once(player_hand)
    player_score_probabilities = get_score_probabilities(player_hand)

    print("Dealer score probabilities:")
    print(dealer_score_probabilities)

    win, push, lose = 0.0, 0.0, 0.0

    for score, chance in player_score_probabilities.items():
        win += chance * dealer_score_probabilities[dealer_score_probabilities.index < score].sum()
        push += chance * dealer_score_probabilities[dealer_score_probabilities.index == score].sum()
        lose += chance * dealer_score_probabilities[dealer_score_probabilities.index > score].sum()

    print(pd.DataFrame([win,push,lose], index=["win","push","lose"], columns=["probability"]).T)
    print("Expected return of hitting once is", win - lose)

    return win - lose

In [17]:
deck = get_deck()
player = get_hand(deck, [10,6])
dealer = get_hand(deck, [7])

r_stand = get_expected_return_standing(deck, player, dealer)
r_hit = get_expected_return_hitting_once(deck, player, dealer)

print("")
print("Expected return of standing is", r_stand)
print("Expected return of hitting once is", r_hit)

Calculating expected return of standing with the following hand:
player    chance  score  cards  aces     2     3     4     5     6     7     8     9  \
0     1.0   16.0    2.0   0.0  24.0  24.0  24.0  24.0  23.0  24.0  24.0  24.0   

     10    11  
0  95.0  24.0  
dealer    chance  score  cards  aces     2     3     4     5     6     7     8     9  \
0     1.0    7.0    1.0   0.0  24.0  24.0  24.0  24.0  24.0  23.0  24.0  24.0   

     10    11  
0  96.0  24.0  
Dealer score probabilities:
0     0.261936
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    0.000000
16    0.000000
17    0.369208
18    0.137931
19    0.078428
20    0.078682
21    0.073816
22    0.000000
dtype: float64
                  win  push      lose
probability  0.261936   0.0  0.738064
Expected return of standing is -0.47612864826358214
Calculating exp